# TD de Machine Learning : `Prédiction de la Note moyenne des livres`
### Exercice 1 :
Les données d'entrées sont sauvegardées dans l'index `books` sur Elasticsearch.
Créer une base de données d'entraînement et de test.
Vous enregistrerez la base d'entraînement dans un index d'elasticsearch, qui portera le nom suivant :

- `uppa_2025_train_nom_prenom`


### Exercice 2 :    
Créer un modèle `more_like_this` pour la prédiction de la note moyenne du livre (`average_rating`)

### Exercice 3 :     
Créer un modèle `more_like_this` par langue pour la prédiction de la note moyenne du livre (`average_rating`). Vous devrez dans un premier temps identifier la langue de chaque livre.

## Installation des librairies

In [ ]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install elasticsearch-dsl

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=2d5f97ebe1d0254253fab764e59189162c07b2187f3ab02347f608febd8d3d34
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=3ad819a8b82c3d59b0de2959c2345c60ec6a77d2efc531dfc5b5f25de467c8f0
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


## Importation des librairies

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch_dsl import Search
import pandas as pd
import numpy as np
from langdetect import detect
import wget
from google.colab import drive
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

# Exercice 1


In [ ]:
# @title **Connexion à Elasticsearch**

es = Elasticsearch(
    hosts=['http://formakuntza-kibana.hupi.io:9200/'],
    basic_auth=('student', 'gCzOD2pVnCqVpd13')
)

# Tester la connexion :
print(es.info())

<ipython-input-154-51dbcd876948>:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


{'name': 'instance-0000000000', 'cluster_name': '1b2852bea18b4caa8a6e28ec5c6795d0', 'cluster_uuid': '-AjNWCALS2yUXAuZXkVCog', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [ ]:
# @title **Lecture de l'index "books" comme les données d'entrées**

# @title **Afficher l'index "ecommerce"**
index_name = "books"

s = Search(using=es, index=index_name).query("match_all")

# Utilisation de la méthode scan pour récupérer tous les résultats rapidement
results = s.scan()

# Convertir les résultats en DataFrame
data = [hit.to_dict() for hit in results]
df = pd.json_normalize(data)

# Explorer les données
print(f"Show dataframe shape: {df.shape}")
print(f"Show dataframe first 10 rows: \n {df.head()}")

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,id_book
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,1
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,2
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,3


## Diviser les données en ensembles d'entraînement (80%) et de test (20%)

In [ ]:
# Diviser les données en ensembles d'entraînement (80%) et de test (20%)
train_df, test_df = # A compléter
print(f"Taille de l'ensemble d'entraînement (train_df) : {train_df.shape}")
print(f"Taille de l'ensemble de test (test_df) : {test_df.shape}")

Taille de l'ensemble d'entraînement (train_df) : (5448, 13)
Taille de l'ensemble de test (test_df) : (1362, 13)


## Ecriture des données dans elasticsearch

In [ ]:
index_name = 'uppa_2025_train_nom_prenom' # A MODIFIER

# Mapping for the index (modify according to your data structure)
mapping = # A compléter

# Create the index with the specified mapping
# A compléter

<ipython-input-157-5d756e7ade22>:21: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=mapping, ignore=400)


ObjectApiResponse({'error': {'root_cause': [{'type': 'mapper_parsing_exception', 'reason': 'No handler for type [int] declared on field [id_book]'}], 'type': 'mapper_parsing_exception', 'reason': 'Failed to parse mapping: No handler for type [int] declared on field [id_book]', 'caused_by': {'type': 'mapper_parsing_exception', 'reason': 'No handler for type [int] declared on field [id_book]'}}, 'status': 400})

In [ ]:
def df_to_es(df):
    for record in df.to_dict(orient='records'):
        yield {
            "_index": index_name,
            "_source": record
        }

# Bulk insert the data into Elasticsearch
df_to_index = # A compléter
success, failed = # A compléter  # Set raise_on_error to False

# Print the result
print(f"Successfully indexed {success} documents.")
print(f"Failed to index {failed} documents.")

# Print details of failed documents
for item in failed:
    print(f"Failed document: {item}")

Successfully indexed 5119 documents.
Failed to index [] documents.


# Exercice 2 :
Prédiction de la note moyenne des livres

### Question 1 :   
Calculer un score pour un livre que vous auriez choisi.
L'objectif est de prendre en main la requête `more_like_this`

### Question 2 :     
Créer un modèle sur la base d'entraînement de `train_df` que vous testerez pour l'ensemble des livres présents dans `test_df`. Vous mettrez également en place des métriques afin d'évaluer et améliorer vos différents modèles.

### Question 3 :     
Créer un modèle par langue sur la base d'entraînement de `train_df` que vous testerez pour l'ensemble des livres présents dans `test_df`. Vous mettrez également en place des métriques afin d'évaluer et améliorer vos différents modèles.

## Question 1 : Détection des similarités avec `more_like_this`
Documentation : https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html

In [ ]:
###############################################   Select an item for evaluation

response = # A compléter

print(f"Response: {response} \n")
hits = # A compléter
source_list = # A compléter
df = # A compléter
print(f"Dataframe view: \n{df}")

Response: {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_shard': '[books_index][0]', '_node': '4Auo0ZvbTNWkw58ClqENLQ', '_index': 'books_index', '_id': 'bqqUcZIBDDcGfhtK2z8z', '_score': 1.0, '_ignored': ['description.keyword'], '_source': {'id_book': 6603, 'title': 'The Gods Themselves', 'authors': 'Isaac Asimov', 'categories': 'Science fiction', 'description': 'In the year 2100, mankind on Earth, settlers in a lunar colony and aliens from the para-universe, a strange universe parallel in time to our own, are faced with a race against time to prevent total destruction of the Earth. The invention of the Inter-Universe Electron Pump has threatened the rate of hydrogen fusion in the sun, leading, inevitably, to the possibilty of a vast explosion -- and the vapourisation of the Earth exactly eight minutes later . . .', 'published_year': 2000.0, 'average_rati

In [ ]:
df["description"].iloc[0] # to copy the content of the description

'In the year 2100, mankind on Earth, settlers in a lunar colony and aliens from the para-universe, a strange universe parallel in time to our own, are faced with a race against time to prevent total destruction of the Earth. The invention of the Inter-Universe Electron Pump has threatened the rate of hydrogen fusion in the sun, leading, inevitably, to the possibilty of a vast explosion -- and the vapourisation of the Earth exactly eight minutes later . . .'

In [ ]:
###############################################   Apply the more_like_this function

response = # A compléter

print(f"Response: {response} \n")
hits = # A compléter
source_list = # A compléter
df = # A compléter
print(f"Dataframe view: \n{df}")

Response: {'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4971, 'relation': 'eq'}, 'max_score': 18.599382, 'hits': [{'_shard': '[books_index][0]', '_node': '4Auo0ZvbTNWkw58ClqENLQ', '_index': 'books_index', '_id': 'bqqUcZIBDDcGfhtK2z8z', '_score': 18.599382, '_ignored': ['description.keyword'], '_source': {'id_book': 6603, 'title': 'The Gods Themselves', 'authors': 'Isaac Asimov', 'categories': 'Science fiction', 'description': 'In the year 2100, mankind on Earth, settlers in a lunar colony and aliens from the para-universe, a strange universe parallel in time to our own, are faced with a race against time to prevent total destruction of the Earth. The invention of the Inter-Universe Electron Pump has threatened the rate of hydrogen fusion in the sun, leading, inevitably, to the possibilty of a vast explosion -- and the vapourisation of the Earth exactly eight minutes later . . .', 'published_year': 2000.0

In [ ]:
# explanation of similarities
response["hits"]["hits"]

[{'_shard': '[books_index][0]',
  '_node': '4Auo0ZvbTNWkw58ClqENLQ',
  '_index': 'books_index',
  '_id': 'C_13cZIB5gnEiKXDfa-x',
  '_score': 18.599382,
  '_source': {'title': 'The Gods Themselves',
   'authors': 'Isaac Asimov',
   'categories': 'Science fiction',
   'description': 'In the year 2100, mankind on Earth, settlers in a lunar colony and aliens from the para-universe, a strange universe parallel in time to our own, are faced with a race against time to prevent total destruction of the Earth. The invention of the Inter-Universe Electron Pump has threatened the rate of hydrogen fusion in the sun, leading, inevitably, to the possibilty of a vast explosion -- and the vapourisation of the Earth exactly eight minutes later . . .',
   'published_year': 2000.0,
   'average_rating': 4.09,
   'num_pages': 288.0,
   'ratings_count': 41304.0},
  '_explanation': {'value': 18.599382,
   'description': 'sum of:',
   'details': [{'value': 0.23543184,
     'description': 'weight(description:a

Le livre le plus similaire selon le contenu, l'auteur et le titre est le livre "The Future of the Universe" qui est noté à 3.75. Nous aurions donc fait une erreur de : 4.09 - 3.75 = 0.34

### Correction : pour aller plus loin
Recherche des termes avec un second modèle `multi_match`
Documentation : https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html

In [ ]:
###############################################    Apply the multi_match function

response = # A compléter

print(response)
hits = # A compléter
source_list = # A compléter
df = # A compléter
print(df)

{'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5062, 'relation': 'eq'}, 'max_score': 271.19333, 'hits': [{'_index': 'books_index', '_id': 'bqqUcZIBDDcGfhtK2z8z', '_score': 271.19333, '_ignored': ['description.keyword'], '_source': {'id_book': 6603, 'title': 'The Gods Themselves', 'authors': 'Isaac Asimov', 'categories': 'Science fiction', 'description': 'In the year 2100, mankind on Earth, settlers in a lunar colony and aliens from the para-universe, a strange universe parallel in time to our own, are faced with a race against time to prevent total destruction of the Earth. The invention of the Inter-Universe Electron Pump has threatened the rate of hydrogen fusion in the sun, leading, inevitably, to the possibilty of a vast explosion -- and the vapourisation of the Earth exactly eight minutes later . . .', 'published_year': 2000.0, 'average_rating': 4.09, 'num_pages': 288.0, 'ratings_count': 41304.0}}, {

## Question 2

In [ ]:
def get_similar_document(row):
    # Extraire les informations pertinentes de la ligne du dataframe
    title = # A compléter
    author = # A compléter
    description = # A compléter

    # Requête more_like_this pour Elasticsearch
    response = # A compléter

    # Récupérer les documents similaires
    hits = # A compléter

    if hits:
        # Extraire les informations du premier résultat le plus pertinent
        most_similar_doc = # A compléter
        # Renvoyer la valeur de la colonne 'average_rating'
        return most_similar_doc.get('average_rating', None)
    else:
        return None

In [ ]:
test_df['similar_average_rating'] = # A compléter

In [ ]:
# compute metrics
mae = # A compléter
print(f"Erreur absolue moyenne (MAE) : {mae}")

mse = # A compléter
print(f"Erreur quadratique moyenne (MSE) : {mse}")

rmse = # A compléter
print(f"Racine de l'erreur quadratique moyenne (RMSE) : {rmse}")

Erreur absolue moyenne (MAE) : 0.28680250783699063
Erreur quadratique moyenne (MSE) : 0.17300156739811912
Racine de l'erreur quadratique moyenne (RMSE) : 0.4159345710542935


## Question 3

### Détection de la langue

In [ ]:
# Fonction pour détecter la langue d'un titre de livre
def detect_language(description):
    try:
        return detect(description)
    except:
        return 'unknown'

In [ ]:
# Ajouter une colonne 'language' au DataFrame
train_df['language'] = # A compléter
test_df['language'] = # A compléter

In [ ]:
train_df.head(2)

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,id_book,language
4235,9780704102477,0704102471,The Egg and I,NaN,Betty MacDonald,Farm life,http://books.google.com/books/content?id=W3dGP...,NaN,1992.0,3.93,288.0,6223.0,4236,unknown
6602,9781857989342,1857989341,The Gods Themselves,NaN,Isaac Asimov,Science fiction,http://books.google.com/books/content?id=8ClIP...,"In the year 2100, mankind on Earth, settlers i...",2000.0,4.09,288.0,41304.0,6603,en


In [ ]:
# Vérifier la distribution des langues
print(train_df['language'].value_counts())

language
en         5198
unknown     220
de           11
id            7
nl            3
da            2
tl            2
vi            1
et            1
it            1
ja            1
ro            1
Name: count, dtype: int64


In [ ]:
for index, row in train_df.iterrows():
    id_book = # A compléter # ID du document dans Elasticsearch
    language = # A compléter  # Langue à mettre à jour

    # Utiliser update_by_query pour mettre à jour le champ 'language'
    update_query = # A compléter

    # Exécuter la requête
    response = # A compléter

    print(f"Updated language for id_book {id_book}: {language}. Response: {response}")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Updated language for id_book 606: en. Response: {'took': 5, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Updated language for id_book 1195: en. Response: {'took': 5, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Updated language for id_book 2875: en. Response: {'took': 6, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Updated 

### Evaluation de la similarité

In [ ]:
def get_similar_document_by_language(row):
    # Extraire les informations pertinentes de la ligne du dataframe
    title = # A compléter
    authors = # A compléter
    description = # A compléter
    language = # A compléter

    # Requête more_like_this pour Elasticsearch, en incluant un filtre sur la langue
    response = # A compléter

    # Récupérer les documents similaires
    hits = response['hits']['hits']

    if hits:
        # Extraire les informations du premier résultat le plus pertinent
        most_similar_doc = # A compléter
        # Renvoyer la valeur de la colonne 'average_rating'
        return most_similar_doc.get('average_rating', None)
    else:
        return None

In [ ]:
test_df['lang_similar_average_rating'] = # A compléter

# Afficher le dataframe avec la nouvelle colonne 'lang_similar_average_rating'
print(test_df.head())

             isbn13      isbn10                                        title  \
1632  9780312994815  0312994818                                  The Firefly   
3891  9780671823498  0671823493                               The Good Earth   
2115  9780375725036  0375725032        Elect Mr. Robinson for a Better World   
2404  9780393320923  0393320928  "What Do You Care what Other People Think?"   
3260  9780517101353  0517101351    A New Collection of Three Complete Novels   

                                       subtitle  \
1632                                    A Novel   
3891                                        NaN   
2115                                        NaN   
2404  Further Adventures of a Curious Character   
3260                                        NaN   

                                      authors                 categories  \
1632                         P. T. Deutermann                    Fiction   
3891                 Pearl Sydenstricker Buck               

In [ ]:
# compute metrics
mae = # A compléter
print(f"Erreur absolue moyenne (MAE) : {mae}")

mse = # A compléter
print(f"Erreur quadratique moyenne (MSE) : {mse}")

rmse = # A compléter
print(f"Racine de l'erreur quadratique moyenne (RMSE) : {rmse}")

Erreur absolue moyenne (MAE) : 0.28711917916337804
Erreur quadratique moyenne (MSE) : 0.17635801104972376
Racine de l'erreur quadratique moyenne (RMSE) : 0.41995001017945427
